<img style="float: center;" src="./images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Brian Kim, Avishek Kumar, and Jonathan Morgan. 

_Citation to be updated on export_

# Data Preparation for Machine Learning - Creating Labels
----

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Data-Preparation-for-Machine-Learning---Creating-Labels" data-toc-modified-id="Data-Preparation-for-Machine-Learning---Creating-Labels-1">Data Preparation for Machine Learning - Creating Labels</a></span><ul class="toc-item"><li><span><a href="#Python-Setup" data-toc-modified-id="Python-Setup-1.1">Python Setup</a></span></li><li><span><a href="#Creating-Labels" data-toc-modified-id="Creating-Labels-1.2">Creating Labels</a></span><ul class="toc-item"><li><span><a href="#Outcome-example:-employed-1-year-after-graduation" data-toc-modified-id="Outcome-example:-employed-1-year-after-graduation-1.2.1">Outcome example: employed 1 year after graduation</a></span></li><li><span><a href="#Repeating-the-Label-Creation-Process" data-toc-modified-id="Repeating-the-Label-Creation-Process-1.2.2">Repeating the Label Creation Process</a></span></li><li><span><a href="#Writing-a-Function-to-Create-Labels" data-toc-modified-id="Writing-a-Function-to-Create-Labels-1.2.3">Writing a Function to Create Labels</a></span></li></ul></li></ul></li></ul></div>

## Python Setup
- Back to [Table of Contents](#Table-of-Contents)

Before we begin, run the code cell below to initialize the libraries we'll be using in this assignment. We're already familiar with `numpy`, `pandas`, and `psycopg2` from previous tutorials. Here we'll also be using [`scikit-learn`](http://scikit-learn.org) to fit modeling.

In [ ]:
%pylab inline
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import time

In [ ]:
# and set our database connection parameters
db_name = "appliedda"
hostname = "10.10.2.10"

## Creating Labels

Labels are the dependent variables, or *Y* variables, that we are trying to predict. In the machine learning framework, your labels are usually *binary*: true or false, often encoded as 1 or 0. 

It is important to clearly and explicitly define the rows (aka observations) of your analysis to ensure you properly combine input datasets and populate the columns (aka features).

In [ ]:
# set database connections - use psycopg2 to more easily execute queries without returning data 
# (eg for series of CREATE queries)
conn = psycopg2.connect(database=db_name, host=hostname)
cursor = conn.cursor()

In [ ]:
# columns we'll use to define the "primary degree earned" date
deg_term = ['degcert_au_inst1_term_earned_1', 'degcert_sm_inst1_term_earned_1',
           'degcert_sp_inst1_term_earned_1', 'degcert_wi_inst1_term_earned_1']

degterm = ','.join([c+'::text' for c in deg_term])

In [ ]:
start_time = time.time()
sql = '''
DROP TABLE cohort_2009;
CREATE TEMP TABLE cohort_2009 AS
SELECT DISTINCT ON (key_id) key_id, format('%s-%s-1', deg_year, deg_term::int*3-2)::date yr_q,
    0 AS label --placeholder for the 'employed' outcome to be created
FROM (
    SELECT key_id, unnest(array[{DEGTERM}]) deg_term, file_year AS deg_year
    FROM data_ohio_olda_2018.oh_hei
    WHERE file_year = 2009 
) q
WHERE deg_term IS NOT NULL
ORDER BY key_id, deg_term, deg_year;

COMMIT;
'''.format(DEGTERM = degterm)
# df = pd.read_sql(sql, conn)
cursor.execute(sql)

print('query complete in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
sql = '''
SELECT *
FROM cohort_2009
'''
df = pd.read_sql(sql, conn)

In [ ]:
print('there are {:,.0f} graduates in our selected study period'.format(df.shape[0]))

In [ ]:
df['key_id'].nunique()

### Outcome example: employed 1 year after graduation

Above we defined our population: individuals who graduated from one of the ~40 public higher ed institutions in 2009.

Now we'll say a given individual is "employed" if they were paid by the same employer for a full quarter one year after graduating

In [ ]:
start_time = time.time()

sql = '''
CREATE TEMP TABLE cohort_2009_jobs_1yr AS
SELECT *, format('%s-%s-1', year, quarter*3-2)::date job_yrq
FROM data_ohio_olda_2018.oh_ui_wage_by_employer a
WHERE (year = 2010 OR (year = 2009 AND quarter = 4) 
        OR (year = 2011 AND quarter = 1))
    AND key_id IN (SELECT key_id FROM cohort_2009);

COMMIT;
'''
cursor.execute(sql)

print('query complete in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
# Load the jobs into Pandas 
sql = '''
SELECT count(*) jq
FROM cohort_2009_jobs_1yr
'''
df = pd.read_sql(sql, conn)
print('people in our cohort were paid for {:,.0f} "job-quarters" from 2009q4 to 2011q1 (inclusive)'.format(df['jq'][0]))

We will now update the `label` variable  to `1` if the individual was employed for one full quarter by the same employer a year after graduating

In [ ]:
start_time = time.time()
# find the full employment quarters
sql = """
CREATE TEMP TABLE cohort_2009_job_full_qtr AS
SELECT a.*
from cohort_2009_jobs_1yr a, cohort_2009_jobs_1yr b, cohort_2009_jobs_1yr c
where a.key_id=b.key_id AND a.employer=b.employer
    AND a.key_id=c.key_id AND a.employer=c.employer
    AND a.job_yrq = (b.job_yrq - '3 month'::interval)::date
    AND a.job_yrq = (c.job_yrq + '3 month'::interval)::date;
    
commit;

"""
cursor.execute(sql)
print('query complete in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
# update label column in the cohort table
sql = """
UPDATE cohort_2009 a SET label = 1
FROM cohort_2009_job_full_qtr b
WHERE a.key_id = b.key_id
    AND a.yr_q = (b.job_yrq -'1 year'::interval)::date
;
    
commit;
"""
cursor.execute(sql)

df = pd.read_sql("SELECT * FROM cohort_2009", conn)

In [ ]:
df.shape

In [ ]:
pd.crosstab(index = df['label'], columns =  'count')

In [ ]:
# or use .value_counts(normalize=True) to show ratio
df['label'].value_counts(normalize=True)

### Repeating the Label Creation Process

We will need at least one (but preferably many) training and test sets for our machine learning analysis. We will put the above steps into a function with parameters for easier reuse.

### Writing a Function to Create Labels

In the above, the SQL queries were all hard coded. In ths section, we demonstrate how to use functions with parameters for the choices we made to define our observations (rows) and label (outcome variable). 

In [ ]:
table_prefix = 'z_' # to "namespace" the table(s) created, recommend team number (eg 't2_')

In [ ]:
def generate_labels(YEAR, year_ahead=1, prefix=table_prefix, DEGterm=degterm, overwrite=False):
    
    #database connection
    conn = psycopg2.connect(database=db_name, host = hostname) 
    cursor = conn.cursor()
    
    # create full set of queries to create labels
    sql = """
    CREATE TABLE ada_edwork.{tbl_prefix}cohort_{year} AS
    SELECT DISTINCT ON (key_id) key_id, format('%s-%s-1', deg_year, deg_term::int*3-2)::date yr_q,
        0 AS label --placeholder for the 'employed' outcome to be created
    FROM (
        SELECT key_id, unnest(array[{DEGTERM}]) deg_term, file_year AS deg_year
        FROM data_ohio_olda_2018.oh_hei
        WHERE file_year = {year} 
    ) q
    WHERE deg_term IS NOT NULL
    ORDER BY key_id, deg_term, deg_year;

    COMMIT;

    CREATE TEMP TABLE cohort_jobs AS
    SELECT *, format('%s-%s-1', year, quarter*3-2)::date job_yrq
    FROM data_ohio_olda_2018.oh_ui_wage_by_employer a
    WHERE key_id IN (SELECT key_id 
            FROM ada_edwork.{tbl_prefix}cohort_{year})
        AND (year = {year}+{ahead} 
            OR (year = {year} AND quarter = 4) 
            OR (year = {year}+{ahead}+1 AND quarter = 1)
            );

    COMMIT;
    
    CREATE TEMP TABLE cohort_job_full_qtr AS
    SELECT a.*
    from cohort_jobs a, cohort_jobs b, cohort_jobs c
    where a.key_id=b.key_id AND a.employer=b.employer
        AND a.key_id=c.key_id AND a.employer=c.employer
        AND a.job_yrq = (b.job_yrq - '3 month'::interval)::date
        AND a.job_yrq = (c.job_yrq + '3 month'::interval)::date;
    
    COMMIT;
    

    UPDATE ada_edwork.{tbl_prefix}cohort_{year} a SET label = 1
    FROM cohort_job_full_qtr b
    WHERE a.key_id = b.key_id
    AND a.yr_q = (b.job_yrq -'1 year'::interval)::date;


    commit;
    
  
    """.format(year=YEAR, tbl_prefix=prefix, ahead=year_ahead, DEGTERM=DEGterm)
    
    
    # Let's check if the table already exists:
    # This query will return an empty table (with no rows) if the table does not exist
    cursor.execute('''
    SELECT * FROM pg_tables 
    WHERE tablename = '{tbl_prefix}cohort_{year}' 
    AND schemaname = 'ada_edwork';
    '''.format(year=YEAR, tbl_prefix=prefix))
    
    # Let's write table if it does not exist (or if overwrite = True)
    if not(cursor.rowcount) or overwrite:
        print("Creating table")
        cursor.execute(sql)
    else:
        print("Table already exists")

    cursor.close()
    
    # Load table into pandas dataframe
    sql = '''
    SELECT * FROM ada_edwork.{tbl_prefix}cohort_{year}
    '''.format(year=YEAR, tbl_prefix=prefix)
    
    df = pd.read_sql(sql, conn)  
    
    return df

Let's test the function with a couple different paramaters:

In [ ]:
start_time = time.time()

# Set parameter(s):
year = 2007

df_test1 = generate_labels(year)
print('Labels generated in {:.2f} seconds'.format(time.time()-start_time))
pd.crosstab(index = df_test1['label'], columns =  'count')

In [ ]:
start_time = time.time()

# Set parameter(s):
year = 2008

df_test2 = generate_labels(year)
print('Labels generated in {:.2f} seconds'.format(time.time()-start_time))
pd.crosstab(index = df_test2['label'], columns =  'count')

In [ ]:
years = [2009, 2010, 2011]

for y in years:
    start_time = time.time()
    
    df_test3 = generate_labels(y)
    
    print('Labels generated in {:.2f} seconds'.format(time.time()-start_time))
    print(pd.crosstab(index = df_test3['label'], columns =  'count'))

In [ ]:
# here's an easy way to compare proportions of outcomes between DataFrames
df_test1['label'].value_counts(normalize=True)

In [ ]:
df_test2['label'].value_counts(normalize=True)

In [ ]:
df_test3['label'].value_counts(normalize=True)